### Load traning data and do one-hot encoding on labels

In [1]:
import csv

# every 30 lines is a different speaker. Each recording is decomposed into 9 arrays of 4 points (9 lines from the original plot)
with open('training_after_PCA_after_taking_5.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

# generate labels for the training 
integer_encoded = []
for i in range(9):
    integer_encoded += [i] * 30

onehot_encoded = []
for elem in integer_encoded:
    letter = [0 for i in range(9)]
    letter[elem] = 1
    onehot_encoded.append(letter)

y_train=onehot_encoded
x_train=data

### Load testing data and do one-hot encoding on labels

In [2]:
import csv

# every 30 lines is a different speaker. Each recording is decomposed into 9 arrays of 4 points (9 lines from the original plot)
with open('testing_after_PCA_after_taking_5.csv', newline='') as csvfile:
    data_test = list(csv.reader(csvfile))


# generate labels for the training 
integer_encoded = [0]*31 + [1]*35 + [2]*88 + [3]*44 + [4]*29 + [5]*24 + [6]*40 + [7]*50 + [8]*29

onehot_encoded = []
for elem in integer_encoded:
    letter = [0 for i in range(9)]
    letter[elem] = 1
    onehot_encoded.append(letter)

y_test=onehot_encoded
x_test=data_test
print(len(y_test))
print(len(x_test))

370
370


### Convert data from str to list of ints

In [3]:
import string

def Convert_to_list_of_ints(data): 
    t=[]
    c=[]
    for elem in data:
        for i in range(9):
            temp = elem[i].strip(string.punctuation)
            temp = temp.replace(" ", "")
            temp = temp.replace("'","")
            temp = list(map(float, temp.split(",")))
            c.append(temp) #concatenate the sub arrays for each of the 9 lines to make one big array
        t.append(c)
        c=[]
    return t

x_train=Convert_to_list_of_ints(x_train)
x_test=Convert_to_list_of_ints(x_test)

print(len(x_train))
print(len(x_test))


270
370


In [4]:
import numpy as np
import random

# shuffle training data
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

## Turn the lists into np.arrays
x_train, y_train = np.array(x_train), np.array(y_train)

## we also have to convert our data into three-dimensional format
x_train = np.reshape(x_train, (x_train.shape[0], 5,9))

In [5]:
# shuffle testing data
b = list(zip(x_test, y_test))
random.shuffle(b)
x_test, y_test = zip(*b)

## Turn the lists into np.arrays
x_test, y_test = np.array(x_test), np.array(y_test)

## we also have to convert our data into three-dimensional format
x_test = np.reshape(x_test, (x_test.shape[0],5,9))

### The model

In [6]:
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, BatchNormalization, Activation
from sklearn.model_selection import KFold
from keras.regularizers import L1L2

def create_model():
    model = Sequential()
##model.add(Bidirectional(LSTM(50, activation='relu',return_sequences=True,kernel_regularizer=L1L2(l1=0.0, l2=0.0)), input_shape=(x_train.shape[1], 9)))
    model.add(LSTM(units=90,activation='tanh', recurrent_activation='sigmoid', return_sequences=False,input_shape=(x_train.shape[1], 9),dropout=0.2,recurrent_dropout=0.2)) 
##model.add(BatchNormalization())
##model.add(LSTM(units=90,activation='relu', recurrent_activation='sigmoid', return_sequences=True,dropout=0.34,recurrent_dropout=0.34))
##model.add(LSTM(units=90, activation='relu', recurrent_activation='sigmoid',return_sequences=False,dropout=0.34,recurrent_dropout=0.34))
    model.add(Dense(units=9))
    model.add(Activation("linear")) 
    opt= keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error',metrics = ['accuracy'])
    return model
#model.fit(x_train, y_train, batch_size=27, epochs=2) #was 150

# saving the model
#model.save('keras_ML1.h5')


#FIRST TEST K-FOLD
def train_and_evaluate_model(model, data, labels, Data, Labels):
    model.fit(data, labels, batch_size=9, epochs=30) #was 150 #training subset
    score = model.evaluate(Data, Labels, verbose=0) #testing subset
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    return score
    # fit and evaluate here.

Test_Loss = []
Test_Accuracy = []
    
from sklearn.model_selection import KFold
#if __name__ == "__main__":
data, labels = x_train, y_train
kf = KFold(n_splits=10) 
i=0
for train, test in kf.split(x_train):
    print("%s %s" % (train, test))
    i=i+1
    print("Running Fold", i,"/", 10)
    model = None # Clearing the NN.
    model = create_model()
    score = train_and_evaluate_model(model, data[train], labels[train], data[test], labels[test])
    Test_Loss.append(score[0])
    Test_Accuracy.append(score[1])
    
model.save('keras_ML1.h5')

Using TensorFlow backend.
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

[ 27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44
  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188
 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224
 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242
 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260
 261 262 263 264 265 266 267 268 269] [ 0  1  2  3 

243/243 [==============================] - 0s 424us/step - loss: 0.0230 - accuracy: 0.9588
Epoch 28/30
243/243 [==============================] - 0s 373us/step - loss: 0.0231 - accuracy: 0.9794
Epoch 29/30
243/243 [==============================] - 0s 385us/step - loss: 0.0252 - accuracy: 0.9671
Epoch 30/30
243/243 [==============================] - 0s 368us/step - loss: 0.0226 - accuracy: 0.9630
Test loss: 0.013216836377978325
Test accuracy: 0.9629629850387573
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 15

243/243 [==============================] - 0s 377us/step - loss: 0.0259 - accuracy: 0.9630
Epoch 25/30
243/243 [==============================] - 0s 374us/step - loss: 0.0260 - accuracy: 0.9506
Epoch 26/30
243/243 [==============================] - 0s 392us/step - loss: 0.0252 - accuracy: 0.9465
Epoch 27/30
243/243 [==============================] - 0s 370us/step - loss: 0.0243 - accuracy: 0.9465
Epoch 28/30
243/243 [==============================] - 0s 383us/step - loss: 0.0251 - accuracy: 0.9547
Epoch 29/30
243/243 [==============================] - 0s 352us/step - loss: 0.0231 - accuracy: 0.9753
Epoch 30/30
243/243 [==============================] - 0s 384us/step - loss: 0.0228 - accuracy: 0.9794
Test loss: 0.01595645770430565
Test accuracy: 1.0
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  

243/243 [==============================] - 0s 389us/step - loss: 0.0257 - accuracy: 0.9588
Epoch 22/30
243/243 [==============================] - 0s 401us/step - loss: 0.0261 - accuracy: 0.9424
Epoch 23/30
243/243 [==============================] - 0s 372us/step - loss: 0.0265 - accuracy: 0.9588
Epoch 24/30
243/243 [==============================] - 0s 382us/step - loss: 0.0251 - accuracy: 0.9506
Epoch 25/30
243/243 [==============================] - 0s 399us/step - loss: 0.0232 - accuracy: 0.9630
Epoch 26/30
243/243 [==============================] - 0s 379us/step - loss: 0.0226 - accuracy: 0.9671
Epoch 27/30
243/243 [==============================] - 0s 389us/step - loss: 0.0230 - accuracy: 0.9753
Epoch 28/30
243/243 [==============================] - 0s 387us/step - loss: 0.0219 - accuracy: 0.9794
Epoch 29/30
243/243 [==============================] - 0s 393us/step - loss: 0.0219 - accuracy: 0.9300
Epoch 30/30
243/243 [==============================] - 0s 396us/step - loss: 0.0224 -

243/243 [==============================] - 0s 420us/step - loss: 0.0281 - accuracy: 0.9300
Epoch 18/30
243/243 [==============================] - 0s 401us/step - loss: 0.0310 - accuracy: 0.9177
Epoch 19/30
243/243 [==============================] - 0s 393us/step - loss: 0.0292 - accuracy: 0.9424
Epoch 20/30
243/243 [==============================] - 0s 399us/step - loss: 0.0278 - accuracy: 0.9547
Epoch 21/30
243/243 [==============================] - 0s 408us/step - loss: 0.0285 - accuracy: 0.9342
Epoch 22/30
243/243 [==============================] - 0s 406us/step - loss: 0.0281 - accuracy: 0.9218
Epoch 23/30
243/243 [==============================] - 0s 413us/step - loss: 0.0282 - accuracy: 0.9465
Epoch 24/30
243/243 [==============================] - 0s 392us/step - loss: 0.0245 - accuracy: 0.9383
Epoch 25/30
243/243 [==============================] - 0s 417us/step - loss: 0.0226 - accuracy: 0.9547
Epoch 26/30
243/243 [==============================] - 0s 401us/step - loss: 0.0252 -

243/243 [==============================] - 0s 426us/step - loss: 0.0328 - accuracy: 0.9300
Epoch 14/30
243/243 [==============================] - 0s 428us/step - loss: 0.0330 - accuracy: 0.9053
Epoch 15/30
243/243 [==============================] - 0s 426us/step - loss: 0.0332 - accuracy: 0.9259
Epoch 16/30
243/243 [==============================] - 0s 454us/step - loss: 0.0307 - accuracy: 0.9383
Epoch 17/30
243/243 [==============================] - 0s 464us/step - loss: 0.0284 - accuracy: 0.9342
Epoch 18/30
243/243 [==============================] - 0s 443us/step - loss: 0.0282 - accuracy: 0.9465
Epoch 19/30
243/243 [==============================] - 0s 443us/step - loss: 0.0299 - accuracy: 0.9342
Epoch 20/30
243/243 [==============================] - 0s 429us/step - loss: 0.0250 - accuracy: 0.9547
Epoch 21/30
243/243 [==============================] - 0s 449us/step - loss: 0.0269 - accuracy: 0.9547
Epoch 22/30
243/243 [==============================] - 0s 433us/step - loss: 0.0268 -

In [7]:
Test_Accuracy

[0.9259259104728699,
 0.9629629850387573,
 1.0,
 1.0,
 0.8888888955116272,
 0.9259259104728699,
 0.9629629850387573,
 0.9629629850387573,
 0.9259259104728699,
 0.9259259104728699]

In [8]:
Test_Loss

[0.02347448095679283,
 0.013216836377978325,
 0.02045896090567112,
 0.01595645770430565,
 0.027624424546957016,
 0.023177094757556915,
 0.016185492277145386,
 0.02106611616909504,
 0.02101689577102661,
 0.019593657925724983]

In [9]:
from keras.models import load_model

# testing the model
model = load_model('keras_ML1.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.051044314616435285
Test accuracy: 0.7810810804367065
